In [9]:
import geopandas as gpd
import pandas as pd

## Open Europe Data

In [10]:
world_boundaries = gpd.read_file("data/world_boundaries.gpkg")

In [11]:
world_boundaries = world_boundaries[["geometry", "name", "iso3"]]

In [12]:
world_boundaries = world_boundaries.rename(
    columns={"name": "Area", "iso3": "Country Code"}
)

## Open pop data

In [13]:
country_pop_df = pd.read_csv(
    "data/API_SP.POP.TOTL_DS2_en_csv_v2_87/API_SP.POP.TOTL_DS2_en_csv_v2_87.csv",
    skiprows=4,
)

In [14]:
country_pop_df = country_pop_df[["Country Code", "2022"]]

## Get FAO data

In [15]:
production_crops_df = pd.read_csv(
    "data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data.csv"
)

/tmp/ipykernel_63046/4000880986.py:1: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  production_crops_df = pd.read_csv(


In [20]:
cheese_df = production_crops_df[production_crops_df["Item Code"] == 901]

In [22]:
cheese_df = cheese_df[cheese_df["Element"] == "Production"][["Area", "Y2022"]]

## Merge Data

In [26]:
world_boundaries_with_pop_gdf = world_boundaries.merge(
    country_pop_df, on="Country Code", how="left"
)

In [27]:
to_replace = {
    "Turkey": "Türkiye",
    "Bosnia & Herzegovina": "Bosnia and Herzegovina",
    "Moldova, Republic of": "Republic of Moldova",
    "The former Yugoslav Republic of Macedonia": "North Macedonia",
    "Czech Republic": "Czechia",
    "Moldova, Republic of": "Republic of Moldova",
    "Netherlands": "Netherlands (Kingdom of the)",
    "West Bank": "Palestine",
    "U.K. of Great Britain and Northern Ireland": "United Kingdom of Great Britain and Northern Ireland",
}
world_boundaries_with_pop_gdf = world_boundaries_with_pop_gdf.replace(to_replace)

In [28]:
cheese_gdf = world_boundaries_with_pop_gdf.merge(cheese_df, on="Area", how="left")

In [29]:
cheese_gdf = cheese_gdf[["geometry", "Y2022", "Area", "2022"]].fillna(0)
cheese_gdf = cheese_gdf.rename(columns={"Y2022": "CHEESE", "2022": "POPULATION"})
cheese_gdf["CHEESE"] = cheese_gdf["CHEESE"].astype(int)
cheese_gdf["POPULATION"] = cheese_gdf["POPULATION"].astype(int)

In [30]:
cheese_gdf["CHEESE_PER_1000"] = (
    (cheese_gdf["CHEESE"] / (cheese_gdf["POPULATION"] / 1_000)).fillna(0)
).round(decimals=1)
cheese_gdf["CHEESE_PER_100"] = (
    (cheese_gdf["CHEESE"] / (cheese_gdf["POPULATION"] / 100)).fillna(0)
).round(decimals=1)

In [31]:
cheese_gdf.to_file("data/cheese_2022.gpkg")